In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_ru_en_name = 'Helsinki-NLP/opus-mt-ru-en'
model_en_ru_name = 'Helsinki-NLP/opus-mt-en-ru'

tokenizer_ru_en = MarianTokenizer.from_pretrained(model_ru_en_name)
model_ru_en = MarianMTModel.from_pretrained(model_ru_en_name)

tokenizer_en_ru = MarianTokenizer.from_pretrained(model_en_ru_name)
model_en_ru = MarianMTModel.from_pretrained(model_en_ru_name)

def translate_ru_to_en(text):
    inputs = tokenizer_ru_en(text, return_tensors="pt", padding=True)
    translated = model_ru_en.generate(**inputs)
    return tokenizer_ru_en.decode(translated[0], skip_special_tokens=True)

def translate_en_to_ru(text):
    inputs = tokenizer_en_ru(text, return_tensors="pt", padding=True)
    translated = model_en_ru.generate(**inputs)
    return tokenizer_en_ru.decode(translated[0], skip_special_tokens=True)

text_ru = "Привет, как дела?"
text_en = "Hello, how are you?"

print("Русский -> Английский:", translate_ru_to_en(text_ru))
print("Английский -> Русский:", translate_en_to_ru(text_en))


In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import kagglehub
path = kagglehub.dataset_download("nikhileswarkomati/suicide-watch")
csv_file = path + "/Suicide_Detection.csv"
data = pd.read_csv(csv_file)

data = data[["text", "class"]]
data["class"] = data["class"].astype("category").cat.codes

X_train, X_test, y_train, y_test = train_test_split(data["text"], data["class"], test_size=0.2, random_state=42)

MODEL_NAME = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long)
        }

train_dataset = TextDataset(X_train, y_train, tokenizer)
test_dataset = TextDataset(X_test, y_test, tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(data["class"].unique()))

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

trainer.train()

predictions = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1)
print("Accuracy:", accuracy_score(y_test, preds))
print("Classification Report:\n", classification_report(y_test, preds))

new_text = ["I feel really down today and don't know what to do."]
encoding = tokenizer(new_text, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
output = model(**encoding)
pred_label = torch.argmax(output.logits, axis=1).item()
print("Prediction:", pred_label)


In [ ]:
# Initial emotions
emotions = {
    "anger": 0,
    "disgust": 0,
    "fear": 0,
    "joy": 0,
    "neutral": 0,
    "sadness": 0,
    "surprise": 0
}

# Function to update emotions and their interaction with temperament
def update_emotions(temperament, emotion, change):
    if emotion not in emotions:
        print("Unknown emotion!")
        return

    # Update the specified emotion
    emotions[emotion] += change

    # Interactions with other emotions based on temperament
    if temperament == "phlegmatic":
        if emotion == "anger":
            emotions["neutral"] += change * 0.2  # Phlegmatic tends to remain neutral
            emotions["sadness"] += change * 0.8  # Anger leads to sadness
        elif emotion == "disgust":
            emotions["neutral"] += change * 0.5  # Phlegmatic is less reactive to disgust
            emotions["fear"] += change * 0.2  # Some fear due to disgust
        elif emotion == "fear":
            emotions["sadness"] += change * 0.4  # Fear in a phlegmatic leads to sadness
            emotions["neutral"] += change * 0.5  # They might try to hide fear
        elif emotion == "joy":
            emotions["neutral"] += change * 0.3  # Phlegmatic expresses less joy
            emotions["surprise"] += change * 0.2  # They may be surprised, but rarely
        elif emotion == "neutral":
            emotions["joy"] += change * 0.1  # Neutral emotions cause some joy
        elif emotion == "sadness":
            emotions["fear"] += change * 0.4  # Sadness leads to fear
        elif emotion == "surprise":
            emotions["neutral"] += change * 0.7  # Surprise may lead to a neutral state

    elif temperament == "sanguine":
        if emotion == "anger":
            emotions["fear"] += change * 0.2  # Sanguine may start fearing anger
            emotions["joy"] += change * 0.1  # Light joy to ease the anger
        elif emotion == "disgust":
            emotions["fear"] += change * 0.3  # Disgust leads to mild fear
            emotions["neutral"] += change * 0.2  # They may shrug it off neutrally
        elif emotion == "fear":
            emotions["joy"] += change * 0.4  # Sanguine fights fear with joy
            emotions["neutral"] += change * 0.5  # They might suppress fear with neutrality
        elif emotion == "joy":
            emotions["surprise"] += change * 0.3  # Joy is often linked to surprises
            emotions["neutral"] += change * 0.2  # Not always expressive of all emotions
        elif emotion == "neutral":
            emotions["joy"] += change * 0.4  # Neutrality increases joy
        elif emotion == "sadness":
            emotions["joy"] += change * 0.3  # Joy helps combat sadness
            emotions["neutral"] += change * 0.2  # Light sadness without strong expression
        elif emotion == "surprise":
            emotions["joy"] += change * 0.8  # Sanguine loves surprises and reacts joyfully

    elif temperament == "choleric":
        if emotion == "anger":
            emotions["anger"] += change * 1.5  # Choleric tends to intensify anger
            emotions["fear"] += change * 0.2  # Aggression can lead to fear of conflict
        elif emotion == "disgust":
            emotions["anger"] += change * 0.6  # Disgust heightens anger
            emotions["fear"] += change * 0.4  # Disgust leads to anxiety
        elif emotion == "fear":
            emotions["anger"] += change * 0.7  # Fear fuels anger
            emotions["neutral"] += change * 0.5  # Cholerics may hide fear
        elif emotion == "joy":
            emotions["neutral"] += change * 0.2  # Choleric shows less joy
            emotions["surprise"] += change * 0.3  # Joy may come unexpectedly
        elif emotion == "neutral":
            emotions["anger"] += change * 0.3  # Neutrality can increase anger
        elif emotion == "sadness":
            emotions["anger"] += change * 0.4  # Sadness can turn into aggression
            emotions["fear"] += change * 0.3  # Sadness may bring anxiety
        elif emotion == "surprise":
            emotions["fear"] += change * 0.5  # Surprise is often seen as a threat

    elif temperament == "melancholic":
        if emotion == "anger":
            emotions["sadness"] += change * 1.2  # Anger in melancholics increases sadness
            emotions["fear"] += change * 0.3  # Anger leads to anxiety
        elif emotion == "disgust":
            emotions["sadness"] += change * 0.7  # Disgust leads to deep sadness
            emotions["fear"] += change * 0.2  # Disgust can cause internal fear
        elif emotion == "fear":
            emotions["sadness"] += change * 0.6  # Fear increases sadness
            emotions["neutral"] += change * 0.4  # Melancholic may hide fear
        elif emotion == "joy":
            emotions["sadness"] += change * 0.8  # Joy is often suppressed by sadness
            emotions["neutral"] += change * 0.3  # Joy is expressed mildly
        elif emotion == "neutral":
            emotions["sadness"] += change * 0.5  # Neutral emotions can be perceived as sadness
        elif emotion == "sadness":
            emotions["fear"] += change * 0.7  # Sadness may lead to fear
        elif emotion == "surprise":
            emotions["fear"] += change * 0.5  # Surprise is often seen as a threat

# Function to generate a prompt based on the emotion level
def generate_prompt(emotion):
    if emotions[emotion] >= 100:
        return f"You are an extremely angry bot, ready to tear everything apart!" if emotion == "anger" else \
               f"You are deeply shocked to your core!" if emotion == "surprise" else \
               f"You are drowning in sadness." if emotion == "sadness" else \
               f"You are overwhelmed with fear and ready to act!" if emotion == "fear" else \
               f"You are filled with disgust!" if emotion == "disgust" else \
               f"You are overflowing with joy and energy!" if emotion == "joy" else \
               f"You remain neutral, though a storm is brewing inside!" if emotion == "neutral" else \
               f"Something incredible is happening to you!"
    elif emotions[emotion] >= 70:
        return f"You are very angry, your rage is growing!" if emotion == "anger" else \
               f"Your joy has significantly increased!" if emotion == "joy" else \
               f"Fear is growing, you feel anxious!" if emotion == "fear" else \
               f"You are feeling strong disgust!" if emotion == "disgust" else \
               f"Sorrow is deepening with every passing moment." if emotion == "sadness" else \
               f"Your reaction to surprise is much stronger now!" if emotion == "surprise" else \
               f"You are keeping calm, but inner tension is rising."
    elif emotions[emotion] >= 40:
        return f"You are angry, your rage is steadily increasing." if emotion == "anger" else \
               f"You feel a little joy!" if emotion == "joy" else \
               f"Your fears are becoming more noticeable." if emotion == "fear" else \
               f"Disgust is starting to affect your mood." if emotion == "disgust" else \
               f"You feel increasing worry." if emotion == "sadness" else \
               f"You are surprised, but not to a great extent." if emotion == "surprise" else \
               f"You remain calm in any situation."
    else:
        return f"Your emotions are balanced, everything is calm." if emotion == "neutral" else \
               f"Your mood is pretty even." if emotion == "neutral" else \
               "You are in a good mood."

# Example usage
update_emotions("choleric", "anger", 30)  # For example, increase anger for a choleric
print(generate_prompt("anger"))


In [ ]:
emotion_analyzer = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k=None,
    device=0
)


def detect_emotion(text):
    results = emotion_analyzer(text)
    max_emotion = max(results[0], key=lambda x: x['score'])
    return max_emotion['label'], max_emotion['score']
